# Generalización para codigos de Superficie con distancia arbitraria

Se comienza con una generalización para un codigo de superficie. Se puede apreciar para codigos de corta distancia como $d=3$ o $d=5$ en los cuales es sencilla la visualización. Se comienza definiendo que para cualquier codigo de superficie de distacia $d$ se va a tener $d^2$ qubits de datos y $d^2 -1$ qubits ancilares. Con esto se presenta una ditribución para $d=5$ de manera clasica y una propuesta geometrica similar que permite mapear los qubits de manera arbitraria para cualquier $d$.

<div style="display: flex; justify-content: center; gap: 20px;">
  <div style="text-align: center;">
    <img src="./media/Distance5Code.jpeg" width="400"/>
    <p style="font-size: 14px;">Figura 1: Representación Clasica</p>
  </div>
  <div style="text-align: center;">
    <img src="./media/DistribuciónD5Kayro.jpeg" width="400"/>
    <p style="font-size: 14px;">Figura 2: Representación Propuesta</p>
  </div>
</div>
<p>
La imagen anterior da una representación clásica donde se logra mapear cada qubit a una coordenada, sin embargo, esto puede aumentar el tamaño de la grilla innecesariamente, en especial en las aristas. Es por ello que se propone una geometría similar en la cual todos los qubits se ven contenidos en una grilla de tamaño \(2d^2 - 1\). Asimismo, es valioso denotar que los qubits de datos se representan como puntos rojos mientras que los ancilares se representan con puntos turquesa. Las mediciones \(X\) se representan con líneas azules y las \(Z\) con líneas rosas.
</p>

Con esta nueva geometría propuesta se puede desarrolla un mapeo arbitrario para cualquier codigo de longitud $d$ que se presenta a continuacion:

In [6]:
import stim
import math

In [7]:
#Se define la distancia del codigo
d = 5

#Se define cuantos qubits de data se va a tener
data_qubits = d*d
#Cantidad de qubits ancilares
ancilla_qubits = (d*d) - 1

total_qubits = data_qubits + ancilla_qubits

In [8]:
def Qubits_coordinates(total_qubits):
    #Para este apartado se propone una distribución como se ve en la Figura 2
    #Se comienza dos listas las cuales guardaran las ubicaciones en una grilla como la de la figura
    data_coords = []
    ancilla_coords = []
    #Se hace un ciclo que va a recorrer toda la grilla que contiene coordenadas y se define cada punto data o ancilla
    for i in range (total_qubits):
        for j in range (total_qubits):
            #Se puede definir el indice para recorrer la matriz de manera continua
            #Se determinan primero las posiciones de los data qubits
            if i%2 == 0 and j%2 == 0:
                data_coords.append((i,j))
            #Se determinan las posiciones de los ancilla qubits sin tomar en cuenta los bordes
            if i%2 != 0 and j%2 != 0:
                ancilla_coords.append((i,j))
            #Se definen los ancilla en los bordes segun la geometría propuesta en la Figura 2
            #Bordes horizontales
            if i == 0 and 4*j + 3 < total_qubits:
                ancilla_coords.append((i, 4*j + 3)) #Esta ecuación 4j+3 separa los impares 1,5,9.. de los otros 3,7,11
            if i == total_qubits - 1 and 4*j + 1 < total_qubits:
                ancilla_coords.append((i, 4*j + 1))
            #Bordes verticales
            if j == 0 and 4*i + 1 < total_qubits:
                ancilla_coords.append((4*i + 1, j))
            if j == total_qubits - 1 and 4*i + 3 < total_qubits:
                ancilla_coords.append((4*i + 3, j))

    return data_coords,ancilla_coords
#Al llamar esta función cada qubit segun la distribución en la Figura 2 se encuentra asociado a un par ordenado
#Con esto se busca obtener una biyección que relacione los numeros naturales con cada par ordenado siguiendo la numeración clasica de codigo
#Se comienza numerando los qubits de data de 1 a d^2 y los ancillares con numeros de d^2 a total_qubits

In [13]:
#Se propone una función que realice un diccionario que realice una biyyeción par ordenado con un número de qubit
def Qubits_pair_num(data_coords #Lista de pares ordenados referentes a los qubits de datos,
                    ,ancilla_coords #Lista de pares ordenados referentes a los qubits ancilla,
                    ,d #Distancia del código
):
    #Diccionario que guarda par ordenado:numero
    qubits_coords_num = {}
    #Se propone un ordenamiento para los data como suele ser en un surface code
    data_order = sorted(data_coords, key=lambda coord: (coord[0], coord[1]))
    #Se asignan los numeros a los datos ordenados como se quiere
    for idx, coord in enumerate(data_order, start=1):
        qubits_coords_num[coord] = idx

    #Se ordenan las ancillas por pares ordenadas
    ancilla_order = sorted(ancilla_coords, key=lambda coord: (coord[0], coord[1]))
    #Se define el numero donde comienzan los números de los ancilla
    start_ancilla = d*d+1
    #Se asignan los números de qubits_ancilla a los restantes
    for idx, coord in enumerate(ancilla_order, start=start_ancilla):
        qubits_coords_num[coord] = idx

    #Se invierte el diccionario para obtener una funcion que relacione los numeros de qubit a sus coordenadas y posterior a eso poder 
    #Relacionar los numeros de qubit con sus adyacentes
    qubits_num_coords= {num: coord for coord, num in qubits_coords_num.items()}
    
    return qubits_num_coords

In [4]:
#Se define una función que va a clasificar las ancillas segun la geometría de la Figura 2, en donde determina si un ancilla es X o Z
def Class_ancilla(qubits_num_coords #Diccionario que relaciona el número del qubit a su par ordenado
                        , d #Distancia del codigo
                       ):
    #Se define el inicio de los número de qubits ancilares y el final
    start = d*d + 1
    end = 2*d*d - 1
    total_qubits = 2*d - 1

    #Se define un diccionario vacío que va a relacionar el número de qubit con si es un estabilizador X o Z
    tipos_num = {}

    #Se define una función auxiliar que va a clasificar acada qubit individualmente por su posición
    def Stabil_coords(i, j):
        #Se analizan los puntos internos a los bordes del código
        if i % 2 == 1 and j % 2 == 1:
            if i % 4 == 1:
                return 'X' if (j % 4 == 1) else 'Z'
            else:
                return 'X' if (j % 4 == 3) else 'Z'
        #Se analizan los puntos en los bordes de la grilla
        if i == 0:
            return 'X' if (j % 4 == 3) else 'Z'
        if i == total_qubits - 1:
            return 'X' if (j % 4 == 1) else 'Z'
        if j == 0:
            return 'X' if (i % 4 == 1) else 'Z'
        if j == total_qubits - 1:
            return 'X' if (i % 4 == 3) else 'Z'
        return 'Z'

    #Se extraen y se clasifica cada ancila
    for num, (i, j) in qubits_num_coords.items():
        if inicio <= num <= fin:
            tipos_num[num] = Stabil_coords(i, j)

    return tipos_num

In [16]:
#Se define una función de distacia
def Dist(data_q,ancilla_q):
    return math.hypot(p[0] - q[0], p[1] - q[1])

In [3]:
def Ancil_per_data(qubits_num_coords #Diccionario que asocia el numero de qubit a par ordenado
                   , d):
    #Se comienza definiendo el radio donde puedene existir los ancilares para el data
    radio = 2**(0.5)

    #Se definen los qubits de datos y donde comienza y termina la numeración de los ancilares
    data_qubits = d*d
    start  = data_qubits + 1
    end = 2*data_qubits - 1

    #Se define un diccionario que va a relacionar el numero de qubit de data con una lista que representa cada ancilar que le corresponde al data
    ancilla_per_data = {}
    
    # Para cada data qubit
    for data_q in range(1, data_qubits + 1):
        data_posit = qubits_num_coords[data_q]
        vecinos = []

        #Se itera sobre los ancilas
        for ancilla_q in range(start, end + 1):
            ancilla_posit = qubits_num_coords[ancilla_q]
            if Dist(data_posit, ancilla_posit) <= radio:
                vecinos.append(ancilla_q)

        ancil_per_data[data_q] = vecinos

    #Con el diccionario definido de data a ancilla, es más conveniente tener algo que relacione ancilla a data
    data_per_ancilla = {ancilla_q: [] for ancilla_q in range(start, end + 1)}
    for data_q, ancillas in ancilla_per_data.items():
        for ancilla_q in ancillas:
            data_per_ancilla[ancilla_q].append(data_q)
    
    return data_per_ancilla

Con el codigo anterior se logra una generalización para cualquier codigo de distancia $d$; una vez construida la grilla como en la Figura 2 y conociendo los ancilla relacionados a cada data se proponen las gates necesarias para relacionar cada qubit. Es importante saber que al aplicar todas las funciones anteriores tenemos 3 diccionarios los cuales son: 
- qubits_num_coords: Relaciona un número de qubit a un par ordenado, contiene todos lo qubits data y ancilla
- tipos_num: Relaciona el número relacionado a cada qubit ancilla con su respectivo estabilizador X o Z
- data_per_ancilla: Relaciona cada uno de los qubits ancilla con una lista de qubits data que son los que medirían el qubit de data


Con esto se propone una función que realice el circuito cuántico, donde se reinician todos los qubits, luego se realizan las relaciones ancila-data

In [14]:
def Quantum_circuit(tipos_num #Diccionario que relaciona el número ancilla con X o Z
                   , data_per_ancilla #Diccionario que relaciona cada numero de qubit ancilla con una lista de qubits data
                    , d #Distancia del codigo
                    , shots #Cantidad de repeticiones del codigo completo, sirve para hacer estadisticas
                   , rounds #Ciclo completo de medición de estabilizadores. Cada ronda genera sindromes diferentes
                   ):
    #Se definen la cantidad de data y ancilla
    data_qubits = d*d
    ancilla_qubits = (d*d) - 1
    total_qubits = data_qubits + ancilla_qubits
    
    #Se parte generando un circuito vacío
    circuit = stim.Circuit()

    #Se propone un reset masivo para todos los qubits
    circuit.append("R", range(1, total_qubits + 1))
    
    #Para detectar los errores de fase se hace que los qubits de data se encuentren en el estado |+>
    circuit.append("H", range(1, data_qubits + 1))

    #Aca se comienza a definir cada circuito y se empieza un ciclo para las rondas
    for rounds_i in range (rounds):
        for i in tipos_num.keys():
            #Se tratan primero los operadores X
            if tipos_num[i] == "X":
                #Se obtienen la lista de quibits datos asociados
                data_asoc = data_per_ancilla.get(i)
                circuit.append("H", [i])
                #Se aplica a cada qubit la relación
                for k in data_asoc:
                    circuit.append("CNOT", [i,k])
                circuit.append("H", [i])
            #Se tratan los operadores Z
            if tipos_num[i] == "Z":
                #Se obtienen la lista de quibits datos asociados
                data_asoc = data_per_ancilla.get(i)
                #Se aplica a cada qubit la relación
                for k in data_asoc:
                    circuit.append("CNOT", [k,i])
                    
        for ancilla in tipos_num:
            circuit.append("M",[ancilla])
            circuit.append("DETECTOR", [stim.target_rec(-1)], label=f"R{rounds_i}_A{ancilla}")

        #Se propone un if se que encargga de reiniciar las ancillas sin ser la ultima
        if rounds_i < rounds - 1:
            circuit.append("R", list(tipos_num.keys())) 

    #Ponen 2 lineas sugeridas por DS para detección de errores especializados
    sampler = circuit.compile_detector_sampler()
    syndroms = sampler.sample(shots=shots)

    return circuit, syndroms